In [1]:
import joblib
import flask
import pandas as pd
import os

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [2]:
app = flask.Flask("test_service")
dir = "."

In [3]:
@app.route('/titanic/predict/', methods=['POST'])
def predict():
    data = flask.request.json
    print("in service")
    print(data)
    titanic_test = pd.DataFrame(data)
    #print(titanic_test.info()) 
    
    objects = joblib.load(os.path.join(dir,'titanic_v1.pkl') )
    titanic_test['Sex_encoded'] = objects.get('le_sex').transform(titanic_test['Sex'])
    titanic_test['Embarked_encoded'] = objects.get('le_emb').transform(titanic_test['Embarked'])
    titanic_test['Age_encoded'] = objects.get('imp_age').transform(titanic_test[['Age']])
    features = objects.get('features')
    X_test = titanic_test[features]
    result = objects.get('clf').predict(X_test)
    print(result)
    return flask.jsonify(target=str(result)) 
    

In [4]:
if __name__ == '__main__':
       app.run(port=8080)

 * Serving Flask app 'test_service'
 * Debug mode: off


 * Running on http://127.0.0.1:8080
Press CTRL+C to quit


in service
[{'Age': 38, 'PassengerId': 100, 'Sex': 'male', 'Fare': 34.6, 'Pclass': 1, 'Embarked': 'S', 'Parch': 2, 'SibSp': 1, 'Ticket': 'XXX', 'Cabin': 'X'}, {'Age': 38, 'PassengerId': 200, 'Sex': 'female', 'Fare': 38.6, 'Pclass': 1, 'Embarked': 'S', 'Parch': 2, 'SibSp': 1, 'Ticket': 'XXX', 'Cabin': 'X'}]


127.0.0.1 - - [01/Jun/2024 16:00:33] "POST /titanic/predict/ HTTP/1.1" 200 -


[0 1]
